<a href="https://colab.research.google.com/github/sj0054pe/MBTI_Estim/blob/master/src/7A_%E6%8C%87%E6%A8%99%E5%88%86%E9%A1%9E%E3%83%A2%E3%83%87%E3%83%AB_%E4%BA%88%E6%B8%AC%E7%94%A8%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88%E3%81%AE%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>BERT 予測 (テスト) 用データセットの作成</center>


#0. 作業環境の準備

##0.1. google driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##0.2. カレントディレクトリへ移動

In [ ]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/MyDrive/研究室/研究_M2/src


##0.3. 現在時刻の取得

In [ ]:
import datetime
import pytz

def pull_Datetime():
  DATETIME_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  DATETIME_now = DATETIME_now.strftime("%Y-%m-%d")
  #DATETIME_now = DATETIME_now.strftime("%Y/%m/%d %H:%M:%S")
  #print(DATETIME_now)

  return DATETIME_now

print(pull_Datetime())

2022-01-25


##option. データセットの指定

In [ ]:
INPUT_CSV_v5='./Assets/Assets_Output/Tweet_Datasets/v6S7_all_forBERT_2021-11-08_U10sentences.csv'

In [ ]:
#INPUT_CSV_v5='/content/drive/MyDrive/研究室/研究_M2/src/Assets/Assets_Output/Tweet_Datasets/v6S7_forBERT_2021-11-09_U20R10000.csv'

#1. データセットのロード

In [ ]:
import pandas as pd
df_v0=pd.read_csv('%s' % INPUT_CSV_v5, engine='python')

# データの確認
print(f'データサイズ： {df_v0.shape}')

df_v0[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

データサイズ： (242883, 13)


,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
0,あけおめなのよ。ことよろなのよ。,2017-12-30 12:02:23+00:00,INTJ,I,N,T,J
1,CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？,2017-12-29 14:01:24+00:00,INTJ,I,N,T,J
2,中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...,2017-11-26 06:54:54+00:00,INTJ,I,N,T,J
3,Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。,2017-11-19 16:00:34+00:00,INTJ,I,N,T,J
4,すげー！！耐えた！！ #鉄腕DASH,2017-11-19 10:32:29+00:00,INTJ,I,N,T,J


#2. データの加工

##2.1. 必要なデータ（文章とラベル）の抽出

In [ ]:
#df_v0.drop(['notes', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1',  'Unnamed: 0.1.1.1'],axis=1, inplace=True)
df_v0.dropna(how='any', axis=0, inplace=True)

df_v0=df_v0
df_v0[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

NameError: ignored

##2.2. データからリツイートの投稿を除去

In [ ]:
#dfXXX=df['text'].str.extract(r'RT\ @(.*)', expand=True) 
df = df_v0[~df_v0['text'].str.contains(r'RT\ @(.*)')]
#dfXXX = df[~df['text'].str.startswith('RT ')]
df[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

##2.3. @mention・URLの削除

In [ ]:
import re
import numpy as np

def remove_MENTIONandURL(text):
  list_text=re.split(' |\n', text)
  #print(list_text)
  len_list=len(list_text)
  Nan = np.nan
  count_element=0
  element_mention=None
  element_URL=None
  for idx, element in enumerate(list_text):
    if re.match(r'(^)(@)[^a-zA-Z0-9_]*', element):
      element_mention=element
    if element.startswith('https:'):
      element_URL=element
      
  #print(element_mention, element_URL)
  if element_mention==None:
    pass
  else:
    list_text.remove(element_mention)
  if element_URL==None:
    pass
  else:
    list_text.remove(element_URL)

  Z=' '.join(list_text)
  
  return Z

df['text'] = df['text'].map(remove_MENTIONandURL)
df.dropna(how='any', axis=0, inplace=True)
df['text']

In [ ]:
df_v3 = df
#df_v3=df.sample(n=100)
df_v3[['text', 'tweet_time', 'conversation_id','MBTI_Type']].head(5)

In [ ]:
!pip install janome

In [ ]:
from janome.tokenizer import Tokenizer
import collections
from tqdm.notebook import tqdm

import re
import numpy as np

t = Tokenizer()

def remove_ShortText(text):
    one_tweet = "".join(text) #"\n".join(text)
    # print("----------------------------------------")
    # print(one_tweet)
    # print('↓')

    Nan = np.nan
    if len(list(t.tokenize(one_tweet)))> 19 and not one_tweet.startswith("私の性格タイプは"): #私の性格タイプは :
        #print("長い : ", one_tweet)
        #pass
        return one_tweet
    else:
        #print("短い : ", one_tweet)
        return None

df_v3['text'] = df_v3['text'][:].map(remove_ShortText)
df_v3.dropna(how='any', axis=0, inplace=True)
df_v3['text']

In [ ]:
df_v3['text'].head(5)

#3. データの保存

##3.1. 手法4までで作成した投稿を更新

In [ ]:
df_all=df_v3
print(f'データサイズ： {df_all.shape}')

FileName=pull_Datetime()
df_all.to_csv('./Assets/Assets_Output/Tweet_Datasets/v7_all_%s_W20.csv' % FileName)
print('./Assets/Assets_Output/Tweet_Datasets/v7_all_%s_W20.csv' % FileName)

##3.2. データセットv7の作成

In [ ]:
import random
 #データの抽出
df_S=df_all.sample(n=10000)

In [ ]:
FileName=pull_Datetime()
df_S.to_csv('./Assets/Assets_Output/Tweet_Datasets/v7S1_forBERT_%s_W20R10000.csv' % FileName)
print('./Assets/Assets_Output/Tweet_Datasets/v7S1_forBERT_%s_W20R10000.csv' % FileName)

##3.3. 記録用に各指標のユーザー数を表示

In [ ]:
def num_Type_Index(element):
  num_Users_Type=df_S.groupby(['%s' % element])
  num_Users_Type.groups
  num_Users_Type.size()

  print(num_Users_Type.size())
  print()

for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_Type_Index(element)

#Option 最大単語数の計算

In [ ]:
!pip install transformers

!pip install fugashi
!pip install ipadic

In [ ]:
## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

def num_max_words(TYPE_INDEX):
  sentences= df_S['text'].values #sentences = df.sentence.values
  labels= df_S['%s' % TYPE_INDEX].values #labels = df.label.values

  # 最大単語数の確認
  max_len = []
  # 1文づつ処理
  len_sentences = len(sentences)
  for sent in tqdm(sentences):
      # Tokenizeで分割
      token_words = tokenizer.tokenize(sent)
      # 文章数を取得してリストへ格納
      max_len.append(len(token_words))
  # 最大の値を確認
  len_max=max(max_len)
  #len_q0, len_q25, len_q50, len_q75, len_q100=np.percentile(max_len,  [0, 25, 50, 75, 100])
  #len_q75=int(len_q75)
  print('最大単語数: ', max(max_len))

for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_max_words(element)